<a href="https://colab.research.google.com/github/hithesh916/-Resume-Matcher-Project/blob/main/Resume_Matcher_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q scikit-learn spacy sentence-transformers
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [17]:
import spacy
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


In [18]:
# Load spaCy and SentenceTransformer models
nlp = spacy.load("en_core_web_sm")
model = SentenceTransformer("all-MiniLM-L6-v2")  # Lightweight and efficient


Add Resume and Job Description Samples

In [19]:
resume_text = """
Experienced software engineer with expertise in Python, Django, and machine learning.
Developed scalable APIs, performed data analysis, and deployed ML models in production.
Skilled in SQL, cloud computing (AWS), and DevOps.
"""

job_description = """
We are hiring a backend developer with strong experience in Python, Django, and cloud platforms like AWS.
Should have knowledge of RESTful APIs, databases, and machine learning concepts.
"""


In [20]:
# Convert resume and job text to embeddings
resume_vector = model.encode([resume_text])[0]
job_vector = model.encode([job_description])[0]


In [21]:
# Compute cosine similarity
similarity_score = cosine_similarity([resume_vector], [job_vector])[0][0]


Display Matching Insights

In [22]:
# Display result
print(f"✅ Resume match score with job description: {similarity_score * 100:.2f}%")

# Decision logic
if similarity_score > 0.75:
    print("📌 Good match! Candidate is suitable.")
elif similarity_score > 0.50:
    print("⚠️ Partial match. May need upskilling or further screening.")
else:
    print("❌ Low match. Candidate may not be suitable.")


✅ Resume match score with job description: 78.48%
📌 Good match! Candidate is suitable.


In [23]:
!pip install -q PyMuPDF python-docx spacy sentence-transformers scikit-learn
!python -m spacy download en_core_web_sm


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [24]:
import fitz  # PyMuPDF
import docx
import spacy
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files

# Load models
nlp = spacy.load("en_core_web_sm")
model = SentenceTransformer('all-MiniLM-L6-v2')


Define Text Extraction Functions



In [25]:
def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def extract_text_from_docx(path):
    doc = docx.Document(path)
    return "\n".join([para.text for para in doc.paragraphs])


Skill Extraction Using spaCy


In [26]:
def extract_skills(text):
    doc = nlp(text)
    skills = [ent.text for ent in doc.ents if ent.label_ in ("ORG", "PRODUCT", "SKILL", "WORK_OF_ART")]
    return list(set(skills))


Resume Matching Function

In [27]:
def match_resume_to_job(resume_text, job_desc):
    resume_vec = model.encode([resume_text])[0]
    job_vec = model.encode([job_desc])[0]
    score = cosine_similarity([resume_vec], [job_vec])[0][0]
    return score


Upload Multiple Resumes & Process

In [ ]:
uploaded_files = files.upload()
job_description = """We are hiring a backend developer with strong experience in Python, Django, and cloud platforms like AWS. Should have knowledge of RESTful APIs, databases, and machine learning concepts."""

for file_name in uploaded_files:
    ext = os.path.splitext(file_name)[1]
    if ext == ".pdf":
        resume_text = extract_text_from_pdf(file_name)
    elif ext == ".docx":
        resume_text = extract_text_from_docx(file_name)
    else:
        print(f"❌ Unsupported file type: {file_name}")
        continue

    match_score = match_resume_to_job(resume_text, job_description)
    skills = extract_skills(resume_text)

    print(f"\n📄 Resume: {file_name}")
    print(f"✅ Match Score: {match_score * 100:.2f}%")
    print("🧠 Extracted Skills:", skills if skills else "N/A")

    if match_score > 0.75:
        print("📌 Good Match")
    elif match_score > 0.5:
        print("⚠️ Partial Match")
    else:
        print("❌ Low Match")
